<a href="https://colab.research.google.com/github/RyanSolving/BA_Analysis/blob/main/PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
andrewmvd_business_analyst_jobs_path = kagglehub.dataset_download('andrewmvd/business-analyst-jobs')

print('Data source import complete.')


Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/business-analyst-jobs/BusinessAnalyst.csv


In [ ]:
os.walk(andrewmvd_business_analyst_jobs_path)

<generator object _walk at 0x7ebf508d7e10>

# Purpose Statement

This markdown's deligated for database normalization, in aim of practicing Power BI building

In [ ]:
df = pd.read_csv(f'{andrewmvd_business_analyst_jobs_path}/BusinessAnalyst.csv')

In [ ]:
df.head()

,Unnamed: 0,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,0,Business Analyst - Clinical & Logistics Platform,$56K-$102K (Glassdoor est.),Company Overview\n\n\nAt Memorial Sloan Ketter...,3.9,Memorial Sloan-Kettering\n3.9,"New York, NY","New York, NY",10000+ employees,1884,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),"Mayo Clinic, The Johns Hopkins Hospital, MD An...",-1
1,1,1,Business Analyst,$56K-$102K (Glassdoor est.),We are seeking for an energetic and collaborat...,3.8,Paine Schwartz Partners\n3.8,"New York, NY","New York, NY",1 to 50 employees,-1,Company - Private,Venture Capital & Private Equity,Finance,Unknown / Non-Applicable,-1,True
2,2,2,Data Analyst,$56K-$102K (Glassdoor est.),"For more than a decade, Asembia has been worki...",3.6,Asembia\n3.6,"Florham Park, NJ","Florham Park, NJ",501 to 1000 employees,2004,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$5 to $10 million (USD),-1,-1
3,3,3,"Information Security Analyst, Incident Response",$56K-$102K (Glassdoor est.),Job Description Summary\nThe Information Secur...,3.6,BD\n3.6,"Franklin Lakes, NJ","Franklin Lakes, NJ",10000+ employees,1897,Company - Public,Health Care Products Manufacturing,Manufacturing,$10+ billion (USD),"Abbott, Siemens, Baxter",-1
4,4,4,Analyst - FP&A Global Revenue,$56K-$102K (Glassdoor est.),Magnite is the world's largest independent sel...,3.4,Rubicon Project\n3.4,"New York, NY","Los Angeles, CA",201 to 500 employees,2007,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"PubMatic, AppNexus, Index Exchange",True


In [ ]:
#Remove the first column, set_index as index
df.set_index('index')
df.drop(columns='Unnamed: 0',inplace=True)

# Cleaning data

In [ ]:
df.isna().sum()

,0
index,0
Job Title,0
Salary Estimate,0
Job Description,0
Rating,0
Company Name,0
Location,0
Headquarters,0
Size,0
Founded,0


Easy Apply null values could be understand as Not allow for easy apply  
Competitors null values were potentially missing

In [ ]:
df.duplicated().sum()

np.int64(0)

There's no duplicated value

# Check for data format error

## Engineering Salary Estimate

In [ ]:
index = df[df['Salary Estimate'].str.len()<10].index
df.loc[index,:] = df.loc[index,:].apply(lambda row: row.shift(2), axis=1)

In [ ]:
pattern = r"\$(\d+)K-\$(\d+)K\s+\(Glassdoor est\.\)"

df[['Salary Estimate_Min','Salary Estimate_Max']]= df['Salary Estimate'].str.extract(pattern)

## Rating

In [ ]:
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

In [ ]:
#impute -1 value
index = df[df['Rating'] == -1].index
df.loc[index,'Rating'] = df['Rating'].median()

## Company Name

In [ ]:
#text handle
df['Company Name'] = df['Company Name'].str.split("\n",expand=True)[0]

## Location

In [ ]:
df['Location'].value_counts()

,count
Location,
"Jacksonville, FL",392
"San Diego, CA",348
"Houston, TX",340
"Austin, TX",331
"San Antonio, TX",325
...,...
"West Menlo Park, CA",1
"Ponte Vedra Beach, FL",1
"Mayport, FL",1


## Size

In [ ]:
df['Size'] = df['Size'].astype(str)
df['Size'] = np.where((df['Size'] == '-1') | (df['Size'] == 'Unknown'),
                      'Unknown',
                      df['Size'])

## Founded

In [ ]:
df['Founded'] = pd.to_numeric(df['Founded'])
df['Founded'] = np.where(df['Founded']==-1,
                        'Unknown',
                        df['Founded'])

## Type of ownership

In [ ]:
df['Type of ownership'] = np.where(df['Type of ownership']=='-1',
                                  'Unknown',
                                  df['Type of ownership'])

## Industry

In [ ]:
df['Industry'] = np.where(df['Industry']=='-1',
                         'Unknown',
                         df['Industry'])

## Sector

In [ ]:
df['Sector'] = np.where(df['Sector']=="-1",
                       'Unknown',
                       df['Sector'])

## Revenue

In [ ]:
df['Revenue'] = np.where(df['Revenue']=='-1',
                        'Unknown / Non-Applicable',
                        df['Revenue'])

## Job description

In [ ]:
#Preprocessing text data
import spacy
import re
nlp = spacy.load('en_core_web_sm')

In [ ]:
#Clean text
def clean_text(row):
    text = str(row)
    text = text.lower()
    #replace punctuation
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    doc = nlp(text)
    #Lemmatized words
    text = ' '.join([token.lemma_ for token in doc if not token.is_stop])
    return text

In [ ]:
df['Job Description'] = df['Job Description'].apply(clean_text)

In [ ]:
#Generate ID column
df['ID'] = range(1,len(df)+1)

In [ ]:
df = df.drop(columns=['index','Job Title','Competitors','Easy Apply','Salary Estimate'], axis=1)

# Normalization

In [ ]:
company = df[['Company Name','Rating','Headquarters','Size','Founded','Industry','Sector','Revenue']]
listing = df[['ID','Company Name','Location','Salary Estimate_Min','Salary Estimate_Max',]]
description = df[['ID','Job Description']]

In [ ]:
company = company.drop_duplicates()

# Noun Extracter

In [ ]:
def noun_extracter(text):
  doc = nlp(text)
  nouns = [token.text for token in doc if token.pos_ == 'NOUN']
  return nouns

In [ ]:
description['Nouns'] = description['Job Description'].apply(noun_extracter)

<ipython-input-30-fb0e81946b33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  description['Nouns'] = description['Job Description'].apply(noun_extracter)


In [ ]:
description['Nouns_str'] = description['Nouns'].apply(lambda x: ' '.join(x))

<ipython-input-31-2e6dae1deeec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  description['Nouns_str'] = description['Nouns'].apply(lambda x: ' '.join(x))


In [ ]:
# Compute TF IDF for extract specific key requirements
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [ ]:
tfidf_array = vectorizer.fit_transform(description['Nouns_str']).toarray()

In [ ]:
tfidf_df = pd.DataFrame(tfidf_array, columns=vectorizer.get_feature_names_out(),index=description['ID'])

In [ ]:
#Find the threshold to indenfity key skills require by listing
threshold = tfidf_df.sum(axis=1).median() ## Since the distribution of TF IDF value's right skewed hence, to keep the significant specific skill, we choose median as threshold to keep them in the list
joined_keywords = tfidf_df.apply(lambda row: ' '.join(row[row > 0].index), axis=1).reset_index(name='TFIDF')

In [ ]:
joined_keywords.to_csv('TFIDF.csv',index=False)
company.to_csv('company.csv',index=False)
listing.to_csv('listing.csv',index=False)
description.to_csv('description.csv',index=False)
tfidf_df.to_csv('raw_tfidf.csv',index=False)